In [1]:
using Plots, LinearAlgebra, BenchmarkTools

In [2]:
function spectral_radius(A::Matrix{T}) where T<:Number
    eigs = eigvals(A)
    return findmax([abs(x) for x in eigs])[1]
end


spectral_radius (generic function with 1 method)

In [3]:
A = [2 1; 2 -2]
B = [2 4; 1 -3]
println(spectral_radius(A+B), ", ", spectral_radius(A) + spectral_radius(B))
#spectral_radius(A)

6.437171043518958, 6.151051861499603


In [4]:
spectral_radius(B)

3.7015621187164243

In [5]:
eigvals(A)

2-element Vector{Float64}:
 -2.449489742783178
  2.4494897427831788

In [28]:
function jacobi_iteration(
    A::AbstractMatrix, 
    b::Vector, 
    x0::Vector; 
    etol::Number = 1.0e-5,
    Maxiter = 100_000)
    @assert size(A)[1] == size(A)[2] == size(b)[1]
    
    x = similar(x0)

    D = Diagonal(A)
    L = -LowerTriangular(A) .+ D
    U = -UpperTriangular(A) .+ D

    Dinv = inv(D)
    T = Dinv*(L+U) 
    c = Dinv * b
    for i in 1:Maxiter
        x = T*x0 + c
        if norm(x .- x0, Inf)/norm(x, Inf)< etol
            nitter = i
            print(nitter)
            return x
        else 
            x0 = x
        end
    end
    return nothing
end

function gauss_siedel_iteration(
    A::AbstractMatrix, 
    b::Vector, 
    x0::Vector; 
    etol::Number = 1.0e-5, 
    Maxiter = 100_000)
    @assert size(A)[1] == size(A)[2] == size(b)[1]
    
    x = similar(x0)

    D = Diagonal(A)
    L = -LowerTriangular(A) .+ D
    U = -UpperTriangular(A) .+ D

    DLinv = inv(D-L)
    T = DLinv*U 
    c = DLinv * b
    for i in 1:Maxiter
        x = T*x0 + c
        if norm(x .- x0, Inf)/norm(x, Inf)< etol
            nitter = i
            print(nitter)
            return x
        else 
            x0 = x
        end
    end
    return nothing
end


function sor_iteration(
    A::AbstractMatrix, 
    b::Vector, 
    x0::Vector,
    ω::Real; 
    etol::Number = 1.0e-5, 
    Maxiter = 100_000)

    @assert 0.0 < ω < 2
    x = similar(x0)

    D = Diagonal(A)
    L = -LowerTriangular(A) .+ D
    U = -UpperTriangular(A) .+ D

    Dwinv = inv(D-ω * L)
    T = Dwinv * ((1-ω) * D + ω * U)
    c = ω * Dwinv * b
    for i in 1:Maxiter
        x = T*x0 + c
        if norm(x .- x0, Inf)/norm(x, Inf)< etol
            
            nitter = i
            print(nitter)
            return x
        else 
            x0 = x
        end
    end
    return nothing
    
end


sor_iteration (generic function with 1 method)

In [29]:
A = [10.0 -1.0 2.0 0.0; -1 11 -1 3; 2 -1 10 -1; 0 3 -1 8]
b = [6, 25, -11, 15]
x0 = [1, 1, -1, 1]

4-element Vector{Int64}:
  1
  1
 -1
  1

In [30]:
x=jacobi_iteration(A, b, x0, etol=1.0e-5)

13

4-element Vector{Float64}:
  0.9999964637124269
  2.0000051877531875
 -1.0000041868849727
  1.000006667932778

In [31]:
x=gauss_siedel_iteration(A, b, x0, etol=1.0e-5)

5

4-element Vector{Float64}:
  0.9999998365212163
  2.000000918751941
 -1.0000000773611672
  0.9999996457978763

In [34]:
x = sor_iteration(A, b, x0, 0.7, etol=1.0e-5)

13

4-element Vector{Float64}:
  0.9999873334080895
  1.9999843450596808
 -0.999992522758998
  1.0000136119326923

In [11]:
A = [4 3 0; 3 4 -1; 0 -1 4]
b=[-1 ; 4 ;5]

3-element Vector{Int64}:
 -1
  4
  5

In [12]:
D=Diagonal(A)
L = LowerTriangular(A) .- D
U = UpperTriangular(A) .- D

3×3 UpperTriangular{Int64, Matrix{Int64}}:
 0  2  -2
 ⋅  0   1
 ⋅  ⋅   0

In [13]:
spectral_radius(inv(D)*(L+U))

1.0809840389418118e-5

In [14]:
spectral_radius(inv(D+L)U)

2.0